<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Kitti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pykitti